In [1]:
import ipywidgets as widgets
widgets.IntSlider()


IntSlider(value=0)

In [2]:
import networkx as nx
import matplotlib.pyplot as plt
import ipywidgets as widgets
from matplotlib.animation import FuncAnimation 
from IPython.display import HTML, display

# ── Interactive controls ──
n_slider   = widgets.IntSlider(value=5, min=3, max=15, step=1, description='n:')
m_slider   = widgets.IntSlider(value=4, min=2, max=12, step=1, description='m:')
run_button = widgets.Button(description='Run Animation')
output     = widgets.Output()

# ── Graph constructors ──
def build_n_pan(n):
    G = nx.cycle_graph(range(1, n+1))
    G.add_node(0); G.add_edge(0, 1)
    return G

def compute_base_coloring(n):
    base = {}
    if n % 2 == 0:
        for i in range(1, n+1):
            base[i] = 1 if i % 2 else 2
        base[0] = 2
    else:
        r = n % 3
        if r == 0:
            for i in range(1, n+1):
                base[i] = ((i-1) % 3) + 1
            base[0] = 3
        elif r == 1:
            for i in range(1, n):
                base[i] = ((i-1) % 3) + 1
            base[n] = 2; base[0] = 3
        else:
            for i in range(1, n-1):
                base[i] = ((i-1) % 3) + 1
            base[n-1] = 1; base[n] = 2; base[0] = 3
    return base

# ── Animation builder ──
def equitable_anim(n, m):
    G = build_n_pan(n)
    base = compute_base_coloring(n)
    H = nx.path_graph(range(1, m+1))
    GH = nx.cartesian_product(G, H)
    k = max(base.values())
    coloring = {(u,v): ((base[u] + v-1) % k or k) for u,v in GH.nodes()}
    
    pos    = nx.spring_layout(GH, seed=42)
    groups = {r: [node for node,c in coloring.items() if c==r] 
              for r in set(coloring.values())}
    cmap   = {0:'lightgray', 1:'red', 2:'green', 3:'blue'}
    
    fig, ax = plt.subplots(figsize=(6,5))
    plt.close()
    titles = ["Step 0: All Gray",
              "Step 1: Reveal Class 1",
              "Step 2: Reveal Class 2",
              "Step 3: Full Coloring"]
    
    def update(frame):
        ax.clear()
        vis = set()
        if frame>=1: vis |= set(groups.get(1,[]))
        if frame>=2: vis |= set(groups.get(2,[]))
        if frame>=3: vis |= set(groups.get(3,[]))
        colors = [cmap[coloring[n]] if n in vis else cmap[0] for n in GH.nodes()]
        nx.draw(GH, pos, node_color=colors, with_labels=True, ax=ax)
        ax.set_title(f"n={n}, m={m} — {titles[frame]}")
    
    return FuncAnimation(fig, update, frames=4, interval=800, repeat=False)

# ── Hook up the button ──
def on_run(b):
    with output:
        output.clear_output(wait=True)
        ani = equitable_anim(n_slider.value, m_slider.value)
        display(HTML(ani.to_jshtml()))

run_button.on_click(on_run)

# ── Display UI ──
display(widgets.VBox([n_slider, m_slider, run_button, output]))
